In [1]:
import datetime as dt
import pandas as pd
import matplotlib as plt

In [2]:
pip install Lifetimes

Note: you may need to restart the kernel to use updated packages.


In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
from sklearn.preprocessing import MinMaxScaler

- InvoiceNo: Fatura Numarası ( Eğer bu kod C ile başlıyorsa işlemin iptal edildiğini ifade eder )
- StockCode: Ürün kodu ( Her bir ürün için eşsiz )
- Description: Ürün ismi
- Quantity: Ürün adedi ( Faturalardaki ürünlerden kaçar tane satıldığı)
- InvoiceDate:  Fatura tarihi
- Price: Fatura fiyatı ( Sterlin )
- CustomerID: Eşsiz müşteri numarası
- Country: Ülke ismi

---

# Görev 1: BG-NBD ve Gamma-Gamma Modellerini Kurarak 6 Aylık CLTV Tahmini Yapılması

---

### Adım 1: 2010-2011 yıllarındaki veriyi kullanarak İngiltere’deki müşteriler için 6 aylık CLTV tahmini yapınız.

---

In [4]:
df_ = pd.read_excel("online_retail_II.xlsx", sheet_name="Year 2010-2011")

In [5]:
df = df_.copy()

In [6]:
df = df[~df["Invoice"].str.contains("C", na=False)] # tilda işareti ~ koyarak başında c olmyanlar gelidi
df.describe().T
df = df[(df['Quantity'] > 0)]
df.dropna(inplace=True)

- CLTV = satın alma başına ortalama kazanç * satın alma sayısı
- CLTV = (Müşteri Değeri / Kayıp Oranı) * Kar Marjı



 - df["TotalPrice"] = df["Quantity"] * df["Price"]
 - cltv_c["profit_margin"] = df["TotalPrice"] * 0.10
 -

- cltv_c["cltv"] = (cltv_c["customer_value"] / churn_rate) * cltv_c["profit_margin"]

In [7]:
df["TotalPrice"] = df["Quantity"] * df["Price"]

cltv_c = df.groupby('Customer ID').agg({'Invoice': lambda x: x.nunique(),
                                        'Quantity': lambda x: x.sum(),
                                        'TotalPrice': lambda x: x.sum()})

cltv_c.columns = ['total_transaction', 'total_unit', 'total_price']

- Average Order Value (average_order_value = total_price / total_transaction)

In [8]:
cltv_c["average_order_value"] = cltv_c["total_price"] / cltv_c["total_transaction"]
cltv_c.head()

,total_transaction,total_unit,total_price,average_order_value
Customer ID,,,,
12346.00,1,74215,77183.60,77183.60
12347.00,7,2458,4310.00,615.71
12348.00,4,2341,1797.24,449.31
12349.00,1,631,1757.55,1757.55
12350.00,1,197,334.40,334.40


- Purchase Frequency (total_transaction / total_number_of_customers)

In [9]:

cltv_c["purchase_frequency"] = cltv_c["total_transaction"] / cltv_c.shape[0]

In [10]:
cltv_c.shape[0]
cltv_c.head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency
Customer ID,,,,,
12346.00,1,74215,77183.60,77183.60,0.00
12347.00,7,2458,4310.00,615.71,0.00
12348.00,4,2341,1797.24,449.31,0.00
12349.00,1,631,1757.55,1757.55,0.00
12350.00,1,197,334.40,334.40,0.00


- Repeat Rate & Churn Rate (birden fazla alışveriş yapan müşteri sayısı / tüm müşteriler)

In [11]:
repeat_rate = cltv_c[cltv_c["total_transaction"] > 1].shape[0] / cltv_c.shape[0]

churn_rate = 1 - repeat_rate

- Profit Margin (profit_margin =  total_price * 0.10) kar marjı

In [12]:
cltv_c["Profit_Margin"] = df["TotalPrice"] * 0.10 #kar marjı

- Customer Value (customer_value = average_order_value * purchase_frequency)

In [13]:
cltv_c['customer_value'] = cltv_c['average_order_value'] * cltv_c["purchase_frequency"]

- Customer Lifetime Value (CLTV = (customer_value / churn_rate) x profit_margin)

In [14]:
cltv_c["cltv"] = (cltv_c["customer_value"] / churn_rate) * cltv_c["Profit_Margin"]

cltv_c.sort_values(by="cltv", ascending=False).head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,Profit_Margin,customer_value,cltv
Customer ID,,,,,,,,
18102.00,60,64124,259657.30,4327.62,0.01,4.77,59.84,829.03
14298.00,44,58343,51527.30,1171.07,0.01,20.40,11.88,703.59
16440.00,5,2018,3640.67,728.13,0.00,286.80,0.84,698.89
14156.00,55,57885,117379.63,2134.18,0.01,7.56,27.05,593.97
17450.00,46,69993,194550.79,4229.36,0.01,2.52,44.84,328.16


In [15]:
today_date = dt.datetime(2011, 12, 11)

In [16]:
cltv_df = df.groupby('Customer ID').agg(
    {'InvoiceDate': [lambda InvoiceDate: (InvoiceDate.max() - InvoiceDate.min()).days, # recency güncinsinden
                     lambda InvoiceDate: (today_date - InvoiceDate.min()).days],#  T: Müşterinin yaşı.(analiz tarihinden ne kadar süre önce ilk satın alma yapılmış)
     'Invoice': lambda Invoice: Invoice.nunique(), # frequency
     'TotalPrice': lambda TotalPrice: TotalPrice.sum()})

In [17]:
cltv_df.columns = cltv_df.columns.droplevel(0)

In [18]:
cltv_df

,<lambda_0>,<lambda_1>,<lambda>,<lambda>
Customer ID,,,,
12346.00,0,326,1,77183.60
12347.00,365,368,7,4310.00
12348.00,282,359,4,1797.24
12349.00,0,19,1,1757.55
12350.00,0,311,1,334.40
...,...,...,...,...
18280.00,0,278,1,180.60
18281.00,0,181,1,80.82
18282.00,118,127,2,178.05


In [19]:
cltv_df.columns = ['recency', 'T', 'frequency', 'monetary']

In [20]:
cltv_df

,recency,T,frequency,monetary
Customer ID,,,,
12346.00,0,326,1,77183.60
12347.00,365,368,7,4310.00
12348.00,282,359,4,1797.24
12349.00,0,19,1,1757.55
12350.00,0,311,1,334.40
...,...,...,...,...
18280.00,0,278,1,180.60
18281.00,0,181,1,80.82
18282.00,118,127,2,178.05


- Tahminleme yapacağımız için BG-NBD Modelinin Kuralım.

In [21]:
cltv_df.describe().T

cltv_df = cltv_df[(cltv_df['frequency'] > 1)]

In [22]:
from lifetimes import BetaGeoFitter
bgf = BetaGeoFitter(penalizer_coef=0.001)

In [23]:
bgf.fit(cltv_df['frequency'],
        cltv_df['recency'],
        cltv_df['T'])

<lifetimes.BetaGeoFitter: fitted with 2845 subjects, a: 0.12, alpha: 79.82, b: 2.49, r: 2.18>

In [24]:
cltv_df["6_months"] = bgf.predict(1*24,
                                              cltv_df['frequency'],
                                              cltv_df['recency'],
                                              cltv_df['T'])


In [25]:
cltv_df

,recency,T,frequency,monetary,6_months
Customer ID,,,,,
12347.00,365,368,7,4310.00,0.48
12348.00,282,359,4,1797.24,0.31
12352.00,260,297,8,2506.04,0.62
12356.00,302,326,3,2811.43,0.29
12358.00,149,151,2,1168.06,0.42
...,...,...,...,...,...
18272.00,244,247,6,3078.58,0.59
18273.00,255,258,3,204.00,0.36
18282.00,118,127,2,178.05,0.46


In [26]:
cltv_df.groupby("Customer ID").agg({"6_months": "mean"}).sort_values("6_months", ascending=False).head()

,6_months
Customer ID,
12748.00,11.17
14911.00,10.70
17841.00,6.64
13089.00,5.26
14606.00,5.01


- CLTV değerinin yüksek olması, bir müşterinin bir şirket için çok değerli olduğunu gösterir. 
- Bu, müşterinin satın alma sıklığının yüksek olması, harcama miktarının fazla olması ve bir şirketin müşteriye sağladığı değerlerin çok olması gibi nedenlerle olabilir. 
- Yüksek CLTV değerine sahip müşteriler, bir şirket için önemli bir kaynaktır ve bu müşterileri koruma ve memnun etme çabası içinde olmak, bir şirket için önemlidir. Ayrıca, yüksek CLTV değerine sahip müşteriler, bir şirketin uzun vadeli başarısı için önemli bir faktördür.


---

### Adım 2: Elde ettiğiniz sonuçları yorumlayıp, değerlendiriniz.

---

### Yorum:

- Burada  6 aylık ortlama CLTV değerlerini büyükten küçüğe sıraladığımızda CLTV değeri yüksek olan müşterileri kaybetmemek için aşağıdaki bazı şeyleri yapabiliriz.

- Müşteri sadakatini koruma: Müşteri sadakatini koruma, müşterilerin tekrar satın alma olasılığını arttırır. Bu nedenle, müşteri sadakatini koruma programları oluşturulabilir.

- Müşteri memnuniyetini arttırma: Müşteri memnuniyetini arttırma, müşterilerin tekrar satın alma olasılığını arttırır. Bu nedenle, müşteri memnuniyetini arttırmak için çeşitli önlemler alınabilir, örneğin: hızlı ve etkili müşteri hizmetleri, iyi kalite ürünler ve hizmetler, fiyatların uygun olması.

- İletişim: Müşterilerle sürekli ve etkili bir iletişim kurulması, müşteri sadakatini arttırır. Bu nedenle, müşterilere özel promosyonlar, indirimler ve özel teklifler gönderilebilir.

- Özel deneyimler sunma: Müşterilere özel deneyimler sunma, müşteri sadakatini arttırır. Örneğin, müşterilere özel etkinlikler düzenleyebilir veya özel indirimler ve promosyonlar sunabilirsiniz.

---

# Görev 2: Farklı Zaman Periyotlarından Oluşan CLTV Analizi

---

### Adım 1: 2010-2011 UK müşterileri için 1 aylık ve 12 aylık CLTV hesaplayınız

---

In [27]:
cltv_df["1_months"] = bgf.predict(1*4,
                                              cltv_df['frequency'],
                                              cltv_df['recency'],
                                              cltv_df['T'])

In [28]:
cltv_df["12_months"] = bgf.predict(1*48,
                                              cltv_df['frequency'],
                                              cltv_df['recency'],
                                              cltv_df['T'])

In [29]:
cltv_df

,recency,T,frequency,monetary,6_months,1_months,12_months
Customer ID,,,,,,,
12347.00,365,368,7,4310.00,0.48,0.08,0.96
12348.00,282,359,4,1797.24,0.31,0.05,0.63
12352.00,260,297,8,2506.04,0.62,0.10,1.24
12356.00,302,326,3,2811.43,0.29,0.05,0.59
12358.00,149,151,2,1168.06,0.42,0.07,0.83
...,...,...,...,...,...,...,...
18272.00,244,247,6,3078.58,0.59,0.10,1.17
18273.00,255,258,3,204.00,0.36,0.06,0.71
18282.00,118,127,2,178.05,0.46,0.08,0.92


---

### Adım 2: 1 aylık CLTV'de en yüksek olan 10 kişi ile 12 aylık'taki en yüksek 10 kişiyi analiz ediniz.

---

In [30]:
cltv_df.sort_values(by="1_months", ascending=False).head(10)

,recency,T,frequency,monetary,6_months,1_months,12_months
Customer ID,,,,,,,
12748.00,372,374,210,33719.73,11.17,1.87,22.27
14911.00,372,374,201,143825.06,10.70,1.79,21.33
17841.00,371,374,124,40991.57,6.64,1.11,13.24
13089.00,366,370,97,58825.83,5.26,0.88,10.49
14606.00,372,374,93,12156.65,5.01,0.84,9.99
15311.00,373,374,91,60767.90,4.90,0.82,9.78
12971.00,368,373,86,11189.91,4.64,0.78,9.26
14646.00,353,355,74,280206.02,4.18,0.70,8.34
13408.00,371,374,62,28117.04,3.37,0.56,6.73


In [31]:
cltv_df.sort_values(by="12_months", ascending=False).head(10)

,recency,T,frequency,monetary,6_months,1_months,12_months
Customer ID,,,,,,,
12748.00,372,374,210,33719.73,11.17,1.87,22.27
14911.00,372,374,201,143825.06,10.70,1.79,21.33
17841.00,371,374,124,40991.57,6.64,1.11,13.24
13089.00,366,370,97,58825.83,5.26,0.88,10.49
14606.00,372,374,93,12156.65,5.01,0.84,9.99
15311.00,373,374,91,60767.90,4.90,0.82,9.78
12971.00,368,373,86,11189.91,4.64,0.78,9.26
14646.00,353,355,74,280206.02,4.18,0.70,8.34
13408.00,371,374,62,28117.04,3.37,0.56,6.73


---

### Adım 3: Fark var mı? Varsa sizce neden olabilir?

---

In [32]:
customer_analysis = (cltv_df.sort_values(by="1_months", ascending=False).head(10), cltv_df.sort_values(by="12_months", ascending=False).head(10) )

In [33]:
customer_analysis

(             recency    T  frequency  monetary  6_months  1_months  12_months
 Customer ID                                                                  
 12748.00         372  374        210  33719.73     11.17      1.87      22.27
 14911.00         372  374        201 143825.06     10.70      1.79      21.33
 17841.00         371  374        124  40991.57      6.64      1.11      13.24
 13089.00         366  370         97  58825.83      5.26      0.88      10.49
 14606.00         372  374         93  12156.65      5.01      0.84       9.99
 15311.00         373  374         91  60767.90      4.90      0.82       9.78
 12971.00         368  373         86  11189.91      4.64      0.78       9.26
 14646.00         353  355         74 280206.02      4.18      0.70       8.34
 13408.00         371  374         62  28117.04      3.37      0.56       6.73
 18102.00         366  368         60 259657.30      3.31      0.55       6.61,
              recency    T  frequency  monetary  6_

- 12 ve 6 aylık CLTV değerlerini incelediğimizde CLTV değeri yüksek olan müşterilerin ay bazında değerleri değişsede aynı kişiler olduğunu görüyoruz. Sadık müşterilerimiz için müşterilere özel promosyonlar, indirimler ve özel teklifler gibi müşteri kaybını önleme amaçlı şeyler gönderilebilir.

---

# Görev 3: Segmentasyon ve Aksiyon Önerileri

---

### Adım 1: 2010-2011 UK müşterileri için 6 aylık CLTV'ye göre tüm müşterilerinizi 4 gruba (segmente) ayırınız ve grup isimlerini veri setine ekleyiniz. 

---

In [34]:
cltv_df["Segment"] = pd.qcut(cltv_df['6_months'], 4, labels=["D","C"," B","A"])

In [41]:
cltv_df.sort_values(by="monetary", ascending=False).head(10)  # en çok getiri sağlan müşterilerin segmentine bakalım

,recency,T,frequency,monetary,6_months,1_months,12_months,Segment
Customer ID,,,,,,,,
14646.00,353,355,74,280206.02,4.18,0.70,8.34,A
18102.00,366,368,60,259657.30,3.31,0.55,6.61,A
17450.00,359,368,46,194550.79,2.56,0.43,5.10,A
16446.00,204,206,2,168472.50,0.34,0.06,0.67,C
14911.00,372,374,201,143825.06,10.70,1.79,21.33,A
12415.00,313,338,21,124914.53,1.30,0.22,2.59,A
14156.00,361,372,55,117379.63,3.00,0.50,5.99,A
17511.00,370,374,31,91062.38,1.74,0.29,3.47,A
16029.00,335,374,63,81024.84,2.09,0.35,4.17,A


- Not : Burada sıralama yaptığımızda sıralamda yukarıda olan 16446.00 ID numaralı müşternin getirisi iyi olmasına rağmen C sınıfında yer almıştır bunun nedeni frequency ve recency değerlerinin düşük olmasıdır.

---

### Adım 2: 4 grup içerisinden seçeceğiniz 2 grup için yönetime kısa kısa 6 aylık aksiyon önerilerinde bulununuz.

---

In [37]:
cltv_df.groupby("Segment").agg({"count", "mean", "sum"})

recency                    T                frequency             monetary                    6_months             1_months             12_months             
          count     sum   mean count     sum   mean     count   sum  mean    count        sum    mean    count    sum mean    count    sum mean     count     sum mean
Segment                                                                                                                                                               
D           712  110417 155.08   712  214136 300.75       712  1868  2.62      712  705502.76  990.87      712 129.01 0.18      712  21.58 0.03       712  256.99 0.36
C           711  160241 225.37   711  195573 275.07       711  2594  3.65      711 1158382.61 1629.23      711 258.03 0.36      711  43.17 0.06       711  513.85 0.72
 B          711  150162 211.20   711  171346 240.99       711  3785  5.32      711 1438653.84 2023.42      711 378.79 0.53      711  63.41 0.09       711  753.82 1.06
A           711  145066 204.03   711  155186 218.26       711  8795 12.37      711 4992574.96 7021.91      711 758.77 1.07      711 127.11 0.18       711 1509.05 2.12

- Recency(yenilik : analizin yapıldığı tarih- müşterinein son satın aldığı tarih)
- Frequency(müşterinin toplan satın alması)
- Monetary(toplam parasal değer)


- A grubuna sahip müşteriler çok harcama yapan getirisi yüksek olan(Monetary yüksek olanlar) müşterilerdir.Getirisi yüksek olan müşterileri kaybetmemek için, müşteri memnuniyetini artırmaya yönelik çeşitli önlemler alınabilir. Örneğin, hızlı ve etkili müşteri hizmetleri, iyi kalite ürünler ve hizmetler, fiyatların uygun olması gibi.

- D grubuna sahip müşteriler ise getirsi düşük olduğu için şirket için değeri az olan müşterilerdir. Ancak, bu müşteriler hala önemlidir ve onları kaybetmemek için bazı önlemler alınabilir. Örnek olarak:müşterilere özel indirimler ve promosyonlar sunarak onları tekrar satın almaya teşvik edebilirsiniz.